In [1]:
import sys,os,importlib,gc,string
import xarray as xr
import numpy as np
import pandas as pd
from rich.jupyter import print

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

os.chdir('/Users/peterpfleiderer/Projects/tropical_cyclones/tc_emulator/results')

sys.path.append('../scripts')
import _weather_pattern_class; importlib.reload(_weather_pattern_class)

atl = _weather_pattern_class.weather_patterns(source='ERA5', working_directory='./')



In [2]:
atl.load_input('ERA5_VWS-MSLP_noTC3')
all_years = np.array(range(1979,2019))
atl.set_split(years=all_years)

In [3]:
nrows,ncols = 5,4
tag = 'SOM_kmeansInit%sx%s_v1' % (nrows,ncols)
atl.define_plot_environment(pre_mapping='mapping_raw', clustering=tag, post_mapping='mapping_sammon_1979-2018', nrows=nrows, ncols=ncols)
atl.stats_TC(file='tracks/tracks_ibtracks.csv', overwrite=False)

{'SOM': <minisom.MiniSom object at 0x7f80bd3a7f50>}
.//ERA5_VWS-MSLP_noTC3/mapping_raw_1979-2018/SOM_kmeansInit5x4_v1/mapping_sammon_1979-2018/grid_5x4


In [4]:
years = np.arange(1982,2019,1)
# get relative Atl. SST to tropics
sst_MDR_rel = xr.load_dataset('/Users/peterpfleiderer/Projects/data/SST_MDR_mean/OISST_MDR_rel_to_tropics_1982-2019_daily.nc')['sst']
sst_MDR_rel = sst_MDR_rel[np.isin(sst_MDR_rel.time.dt.year,years)]
sst_MDR_rel = sst_MDR_rel[np.isin(sst_MDR_rel.time.dt.month,atl._months['mon'])]
sst_MDR_rel.time.values = np.array([str(d)[:10] for d in sst_MDR_rel.time.values], np.datetime64)
# get sst
sst_MDR = xr.load_dataset('/Users/peterpfleiderer/Projects/data/SST_MDR_mean/OISST_sst_MDR_1981-2019_daily.nc')['sst']
sst_MDR = sst_MDR[np.isin(sst_MDR.time.dt.year,years)]
sst_MDR = sst_MDR[np.isin(sst_MDR.time.dt.month,atl._months['mon'])]
sst_MDR.time.values = np.array([str(d)[:10] for d in sst_MDR.time.values], np.datetime64)
# get sst
sst_tropics = xr.load_dataset('/Users/peterpfleiderer/Projects/data/SST_MDR_mean/OISST_tropics_1982-2019_daily.nc')['sst']
sst_tropics = sst_tropics[np.isin(sst_tropics.time.dt.year,years)]
sst_tropics = sst_tropics[np.isin(sst_tropics.time.dt.month,atl._months['mon'])]
sst_tropics.time.values = np.array([str(d)[:10] for d in sst_tropics.time.values], np.datetime64)
# prepare tracks:
# here ssts are added. this will be needed in the wind component
atl._tracks = atl._tracks.loc[np.isin(atl._tracks.year,years)]
times = np.array([str(d)[:10] for d in atl._tracks.time.values], np.datetime64)
atl._tracks['time'] = np.array([str(d)[:10] for d in atl._tracks.time],np.datetime64)
atl._tracks['sst'] = sst_MDR.loc[times].values
atl._tracks['sst_rel'] = sst_MDR_rel.loc[times].values
atl._tracks['sst_tropics'] = sst_tropics.loc[times].values
atl._tracks['weather_0'] = atl._tracks['label_lag0']
tracks = atl._tracks.loc[np.isfinite(atl._tracks.weather_0)]
tracks = tracks.loc[tracks.distance2land > 0, ['weather_0','sst','sst_rel','sst_tropics','wind','genesis','storm','ACE','year','storm_day','wind_before','wind_change_before','wind_change','month']]

# prepare gensis input
# this is a dataframe with an entry for each day
# this is required to get genesis probabilities
weather_sst = pd.DataFrame()
weather_sst['time'] =  np.array([str(d)[:10] for d in  atl._vector_time.values], np.datetime64)
weather_sst['year'] = atl._vector_time.dt.year
weather_sst['weather_0'] = atl._clust_labels
weather_sst['weather_1'] = np.roll(atl._clust_labels,1)
weather_sst['weather_2'] = np.roll(atl._clust_labels,2)
weather_sst['weather_3'] = np.roll(atl._clust_labels,3)
weather_sst = weather_sst.loc[np.isin(atl._vector_time.dt.year,years)]

genesis = weather_sst.copy()
genesis['genesis'] = [atl._tracks.loc[atl._tracks.time==np.datetime64(tt),'genesis'].sum() for tt in genesis.time]

weather_sst['sst'] = sst_MDR.sel(time=weather_sst.time.values)
weather_sst['sst_tropics'] = sst_tropics.sel(time=weather_sst.time.values)
weather_sst['sst_rel'] = sst_MDR_rel.sel(time=weather_sst.time.values)

genesis['day_in_season'] = 0
weather_sst['day_in_season'] = 0
for year in np.unique(weather_sst.time.dt.year):
    tttmmmppp = weather_sst.loc[(weather_sst.time.dt.year==year),'day_in_season']
    weather_sst.loc[(weather_sst.time.dt.year==year),'day_in_season'] = np.arange(tttmmmppp.shape[0])
    genesis.loc[(genesis.time.dt.year==year),'day_in_season'] = np.arange(tttmmmppp.shape[0])

weather_sst = weather_sst.loc[(weather_sst.day_in_season>=3) & np.isin(weather_sst.year,years)]
genesis = genesis.loc[(genesis.day_in_season>=3) & np.isin(genesis.year,years)]

# train test split by decades
train_test = pd.DataFrame()
train_test['year'] = list(range(1982,2019))
train_test['1982-1988'] = 'train'
train_test.loc[np.isin(train_test.year,np.arange(1982,1989)), '1982-1988'] = 'test'
train_test['1989-1998'] = 'train'
train_test.loc[np.isin(train_test.year,np.arange(1989,1999)), '1989-1998'] = 'test'
train_test['1999-2008'] = 'train'
train_test.loc[np.isin(train_test.year,np.arange(1999,2009)), '1999-2008'] = 'test'
train_test['2009-2018'] = 'train'
train_test.loc[np.isin(train_test.year,np.arange(2009,2019)), '2009-2018'] = 'test'

In [5]:
test_period = '2009-2018'
train_years = train_test.loc[train_test[test_period]=='train', 'year'].values
test_years = train_test.loc[train_test[test_period]=='test', 'year'].values
train_folder = atl._dir_lvl4 + '/emulator/' + str(test_period)+'/'

In [6]:
# get SST changes over the MDR
sst_hist = xr.open_dataset('/Users/peterpfleiderer/Projects/data/SST/HadISST_sst.nc')['sst'].loc[:,20:10,-90:-20]
lat_weight = np.cos(np.deg2rad(sst_hist.latitude.values))
lat_weight_array = np.repeat(lat_weight[np.newaxis,:], sst_hist.shape[2], 0).T
valid = np.isfinite(sst_hist[0,:,:].values)
sst_hist = xr.DataArray(np.nansum(sst_hist * lat_weight_array, axis=(1,2)) / np.sum(lat_weight_array[valid]), coords={'time':sst_hist.time}, dims=['time'])
hadisst_MDR = sst_hist[np.isin(sst_hist.time.dt.month,[8,9,10])].groupby('time.year').mean('time')

# yearly SSTS
tmp = xr.open_dataset('/Users/peterpfleiderer/Projects/data/SST_MDR_mean/OISST_sst_MDR_1981-2019_daily.nc')['sst']
oisst_MDR = tmp.loc[np.isin(tmp.time.dt.month,[8,9,10])].groupby('time.year').mean('time')[1:]

# experiments
plt.close('all')
fig,ax = plt.subplots(nrows = 1)
y = oisst_MDR
import statsmodels.api as sm
lr = sm.OLS(hadisst_MDR.values, sm.add_constant(hadisst_MDR.year.values)).fit()
ax.plot(y.year, y, label='MDR observed', color='k')

detrend = y - (lr.params[1] * y.year + lr.params[0])
shift = lr.params[0] + lr.params[1] * 1982
ax.plot(y.year, detrend + shift, label='MDR 1982 levels', color='c')
shift = lr.params[0] + lr.params[1] * 2018
ax.plot(y.year, detrend + shift, label='MDR 2018 levels', color='m')
shift = lr.params[0] + lr.params[1] * 1900
ax.plot(y.year, detrend + shift, label='MDR 1900 levels', color='g')
ax.legend()
plt.savefig(train_folder + '/SSTs_experiments_oisst_hadisst_trend.png')

In [7]:
import string

In [8]:
# choose components
alphabet = iter(list(string.ascii_lowercase))
comps_todo = [
    {'g':'gWeaLag2Weight', 'sL':'sLWeaNeigh', 'wS':'wS100nnQrSST', 'Emu':'Emu0', 'name':'main','l':next(alphabet),'c':'c'},
    ]
N = 1000
years = range(1982,2019)
overwrite = True
for dt in comps_todo:
    tag = '_'.join([dt[k] for k in ['g','sL','wS','Emu']])
    print(tag)
    import _emulator; importlib.reload(_emulator); from _emulator import *
    for k,v in {k:v for k,v in dt.items() if k in ['g','sL','wS','Emu']}.items():
        exec("import %s; importlib.reload(%s); from %s import *" % tuple(['components.'+k+'.'+v]*3))
    genesis_obj = pickle.load(open(train_folder+'/_comp_g_'+dt['g']+'/genesis_obj.pkl', 'rb'))
    end_obj = pickle.load(open(train_folder+'/_comp_sL_'+dt['sL']+'/end_obj.pkl', 'rb'))
    wind_obj = pickle.load(open(train_folder+'/_comp_wS_'+dt['wS']+'/wind_obj.pkl', 'rb'))
    emu = storm_emulator(dir=train_folder, tag=tag, emulate_season_function=emulate_season_function)
    emu._weather_sst = weather_sst
    emu.get_stats_seasonal_obs(tracks, train_test.year.values)
    counterFacts = {}
    # all decades .....
    for shift_year in [1900,1982,2018,0]:
        for year in years:
            if shift_year != 0:
                sst_diff_counterfactual = float((lr.params[0] + lr.params[1] * shift_year) - (lr.params[0] + lr.params[1] * year))
            else:
                sst_diff_counterfactual = 0
            # print(year, shift_year, sst_diff_counterfactual)
            emu.simulate_sst_counterfactual(name='_%s_shift%s' %(year,shift_year), years=[year], sst_shift=sst_diff_counterfactual, N=N, genesis_obj=genesis_obj, wind_obj=wind_obj, end_obj=end_obj, overwrite=False)
            # merged
            if year == 1982:
                counterFacts['%s SST levels' %(shift_year)] = emu._simu
                counterFacts['%s SST levels' %(shift_year)]['sst'] = xr.DataArray(coords={'year':years}, dims=['year'])
            else:
                for indicator in emu._simu.keys():
                    counterFacts['%s SST levels' %(shift_year)][indicator] = xr.concat((counterFacts['%s SST levels' %(shift_year)][indicator], emu._simu[indicator]), dim='year')

            counterFacts['%s SST levels' %(shift_year)]['sst'].loc[year] = weather_sst.loc[weather_sst.year == year, 'sst'].mean() + sst_diff_counterfactual


gWeaLag2Weight_sLWeaNeigh_wS100nnQrSST_Emu0

In [9]:
counterFacts['%s SST levels' %(1900)].update({'color':'gray', 'lsty':':', 's':-0.3, 'label':'pre-industrial\nSST levels'})
counterFacts['%s SST levels' %(1982)].update({'color':'c', 'lsty':'--', 's':-0.0, 'label':'1982\nSST levels'})
counterFacts['%s SST levels' %(2018)].update({'color':'darkmagenta', 'lsty':'-', 's':0.3, 'label':'2018\nSST levels'})
counterFacts['%s SST levels' %(0)].update({'color':'k', 'lsty':'-', 's':0.1, 'label':'observed\nSSTs'})

In [10]:
    os.system('mkdir '+emu._dir+'/'+emu._tag+'/counter_2009-2018/')


    year_groups = {
        0 : {'years':all_years, 'color':'blue', 'label':'all years', 'lsty':'-', 'eva':0.5, 'short':'all years'},
        1 : {'years':[yr for yr in years if float(counterFacts['2018 SST levels']['ACE'].loc[yr].mean('run').values) >= 126.1], \
                    'color':'red', 'label':'favorable large scale conditions', 'lsty':'--', 'eva':0.3, 'short':'favorable'},
        # 2 : {'years':[yr for yr in all_years if float(counterFacts['all_2009-2018SST'][indicator].loc[yr].mean('run').values) < 126.1], \
                      # 'color':'darkgreen', 'label':'unfavorable large scale conditions', 'lsty':':', 'eva':-0.2, 'short':'un-favorable'},
        # '2017' : {'years':[2017], 'color':'m', 'label':'2017', 'lsty':'-.', 'eva':0.4},
    }
    ##############################
    # seasonal ACE probabilities #
    ##############################
    plt.close('all')
    plt.rc('grid', linestyle="--", color='lightgray')
    fig,axes = plt.subplots(nrows=4, ncols=2, figsize=(8,6), dpi=100, facecolor='w', sharex='col', sharey='row', gridspec_kw={'height_ratios':[5,5,2,2], 'width_ratios':[5,1]}, constrained_layout=True)
    for ax in axes.flatten()[1:]:
        ax.grid(zorder=0)
    indicator = 'ACE'
    axSST = axes[0,0]
    axSST.fill_between([1982,2018],[27.16]*2,[28.62]*2, color='gray', alpha=0.3)
    axM = axes[1,0]
    # tmpCur = counterFacts['all_2009-2018SST'][indicator]
    for scen,DT in counterFacts.items():
        axSST.plot(DT['sst'].year, DT['sst'], color=DT['color'])
        print('%s SST average %s' %(scen, DT['sst'].mean()))

    axL = axes[0,1]
    axL.axis('off')
    for scen in ['%s SST levels' %(yr) for yr in [2018,0,1982,1900]]:
        DT = counterFacts[scen]
        axL.plot([],[], color=DT['color'], label=DT['label'])
    axL.legend(fontsize=8)

    for scen in ['%s SST levels' %(yr) for yr in [2018,1982,1900]]:
        DT = counterFacts[scen]
        tmp = DT[indicator]
        # for q,lsty in zip([50,95],['--','-','-','-.']):
            # axM.plot(tmp.year, np.nanpercentile(tmp, q, axis=1), color=DT['color'], linestyle=lsty)

        for iyr,yr in enumerate(tmp.year.values):
            if DT['sst'][iyr] < 27.16 or DT['sst'][iyr] > 28.62:
                alpha=0.2
            else:
                alpha=1
            pctls = np.nanpercentile(tmp.loc[yr], [5,17,50,83,95])
            x=DT['s'] + yr
            axM.fill_between([x-0.1,x+0.1],[pctls[1]]*2, [pctls[3]]*2, color=DT['color'], alpha=alpha, zorder=3)
            axM.plot([x,x], pctls[[0,-1]], color=DT['color'], alpha=alpha, zorder=3)
            axM.plot([x-0.1,x+0.1],[pctls[2]]*2, color='w', zorder=4)


        for x,yrDT in year_groups.items():
            x += DT['s']
            pctls = np.nanpercentile(tmp.loc[np.isin(tmp.year,yrDT['years'])], [5,17,50,83,95])
            print(scen,yrDT['label'],pctls)
            axes[1,1].fill_between([x-0.1,x+0.1],[pctls[1]]*2, [pctls[3]]*2, color=DT['color'], alpha=alpha, zorder=3)
            axes[1,1].plot([x,x], pctls[[0,-1]], color=DT['color'], alpha=alpha, zorder=3)

        # for yr in tmp.year.values:
        #     axM.scatter([yr]*1000, tmp.loc[yr], color=DT['color'], alpha=0.1, s=7)
    axes[-1,1].set_xticks(list(year_groups.keys()))
    axes[-1,1].set_xticklabels([d['short'] for d in year_groups.values()], rotation=90)
    # axes[0,0].set_xlim(0.9,1.1)

    aces = [[126.1, 2, 'orange','above normal seasons (ACE > 126.1)'], [159.6, 3, 'red','extremely active seasons (ACE > 159.6)']]
    for ace,r,color,name in aces:
        axes[r,1].set_ylim(0,100)
        axes[r,1].set_yticks(np.arange(0,110,25))
        axes[r,0].set_title(name, color=color)
        axes[r,0].set_ylabel('[$\%$]', color=color)
        for ax in axes[r,:]:
            ax.tick_params(axis='y', colors=color)
            for child in ax.get_children():
                if isinstance(child, matplotlib.spines.Spine):
                    child.set_color(color)
        for ax in axes[1,:]:
            ax.axhline(ace, color=color, zorder=2)
        for scen in ['%s SST levels' %(yr) for yr in [2018,1982,1900]]:
            DT = counterFacts[scen]
            tmp = DT[indicator]
            for yr in tmp.year.values:
                y = np.sum(tmp.loc[yr]>ace).values / tmp.loc[yr].values.flatten().shape[0] * 100
                axes[r,0].bar(x=yr,height = y, color=DT['color'], zorder=2)

            for x,yrDT in year_groups.items():
                tmp_ = tmp[np.isin(tmp.year,yrDT['years']),:]
                y = np.sum(tmp_>ace).values / tmp_.values.flatten().shape[0] * 100
                print(ace,scen,yrDT['label'],y)
                axes[r,1].bar(x=x,height = y, color=DT['color'], zorder=2)

    axes[1,0].set_ylim(0,300)
    axes[1,0].set_yticks([0,50,90,126.1,159.6,200,250,300])
    axes[1,0].get_yticklabels()[3].set_color('orange')
    axes[1,0].get_yticklabels()[4].set_color('red')
    axes[1,0].set_ylabel('ACE [m2/s2]')
    axes[-1,0].set_xticks(np.arange(1982,2020,2))
    axes[-1,0].set_xticklabels(np.arange(1982,2020,2), rotation=90)

    for ax,letter in zip(axes.flatten()[[0]+list(range(2,8))],list(string.ascii_lowercase)):
        ax.annotate(letter.upper(), xy=(0,0.95), xycoords='axes fraction', fontweight='bold', fontsize=16, backgroundcolor='w')

    plt.tight_layout()
    plt.savefig(emu._dir+'/'+emu._tag+'/counter_2009-2018/'+indicator+'_yearly_probs.png', bbox_inches='tight', dpi=200); plt.close()


1900 SST levels SST average <xarray.DataArray ()>
array(27.44540243)

1982 SST levels SST average <xarray.DataArray ()>
array(27.84219277)

2018 SST levels SST average <xarray.DataArray ()>
array(28.01639341)

0 SST levels SST average <xarray.DataArray ()>
array(27.92929309)

2018 SST levels all years [ 22.04    40.24    80.16   141.1668 198.922 ]

...

159.6 2018 SST levels favorable large scale conditions 43.93333333333334

159.6 1982 SST levels all years 9.954054054054055

159.6 1982 SST levels favorable large scale conditions 37.983333333333334

159.6 1900 SST levels all years 5.9459459459459465

159.6 1900 SST levels favorable large scale conditions 26.416666666666664

In [93]:
# average ACE diff
for x,yrDT in year_groups.items():
    for scen in ['%s SST levels' %(yr) for yr in [2018,1982,1900]]:
        tmp = counterFacts[scen]['ACE']
        tmp = tmp[np.isin(tmp.year,yrDT['years'])]
        print('%s %s' %(scen, tmp.mean().values))

2018 SST levels 91.46363567567568

1982 SST levels 86.5032627027027

1900 SST levels 75.75406594594595

2018 SST levels 158.05412666666666

1982 SST levels 147.72893333333332

1900 SST levels 130.78388

In [11]:
# percentage of above 126.1 ACE seasons that would not have occurred in other sceanrio
ACE = 159.6
for scen in ['%s SST levels' %(yr) for yr in [2018,1982,1900]]:
    pact = np.sum(counterFacts['%s SST levels' %(2018)]['ACE'] > ACE) / counterFacts['%s SST levels' %(2018)]['ACE'].values.flatten().shape[0]
    ppreind = np.sum(counterFacts[scen]['ACE'] > ACE) / counterFacts[scen]['ACE'].values.flatten().shape[0]
    print(pact.values,ppreind.values)
    print('percentage of above %s ACE seasons that would not have happened in %s %s ' %(ACE, scen, (1 - ppreind.values / pact.values) * 100))

0.11613513513513514 0.11613513513513514

percentage of above 159.6 ACE seasons that would not have happened in 2018 SST levels 0.0

0.11613513513513514 0.09954054054054054

percentage of above 159.6 ACE seasons that would not have happened in 1982 SST levels 
14.289038864323956

0.11613513513513514 0.05945945945945946

percentage of above 159.6 ACE seasons that would not have happened in 1900 SST levels 
48.801489411217126

In [114]:
# chance of 2017 becoming above average in different scenarios
for ACE in [126.1,159.6,225]:
    for scen in ['%s SST levels' %(yr) for yr in [2018,1982,1900]]:
        ppreind = np.sum(counterFacts[scen]['ACE'].loc[2017] > ACE) / counterFacts[scen]['ACE'].loc[2017].values.flatten().shape[0]
        print('percentage of above %s ACE seasons in %s %s ' %(ACE, scen, ppreind.values))

percentage of above 126.1 ACE seasons in 2018 SST levels 0.442

percentage of above 126.1 ACE seasons in 1982 SST levels 0.399

percentage of above 126.1 ACE seasons in 1900 SST levels 0.332

percentage of above 159.6 ACE seasons in 2018 SST levels 0.232

percentage of above 159.6 ACE seasons in 1982 SST levels 0.187

percentage of above 159.6 ACE seasons in 1900 SST levels 0.129

percentage of above 225 ACE seasons in 2018 SST levels 0.042

percentage of above 225 ACE seasons in 1982 SST levels 0.035

percentage of above 225 ACE seasons in 1900 SST levels 0.015

In [55]:
# average ACE 1982-1994
for scen in ['%s SST levels' %(yr) for yr in [0,2018,1982,1900]]:
    DT = counterFacts[scen]
    print('%s 1982:1994 %s' %(scen, DT['ACE'].loc[1982:1994,:].mean().values))
    print('%s %s' %(scen, DT['ACE'].mean().values))


0 SST levels 1982:1994 60.69485538461537

0 SST levels 89.09455243243244

2018 SST levels 1982:1994 63.62867692307692

2018 SST levels 91.46363567567568

1982 SST levels 1982:1994 60.00962461538462

1982 SST levels 86.5032627027027

1900 SST levels 1982:1994 51.93038461538462

1900 SST levels 75.75406594594595

In [14]:
    year_groups = {
        0 : {'years':all_years, 'color':'cyan', 'label':'all years', 'lsty':'-', 'eva':0.5, 'short':'all years'},
        # 1 : {'years':[yr for yr in years if float(counterFacts['2018 SST levels']['ACE'].loc[yr].mean('run').values) >= 126.1], \
        #             'color':'red', 'label':'favorable large scale conditions', 'lsty':'--', 'eva':0.3, 'short':'favorable'},
        2005 : {'years':[2005], 'color':'r', 'label':'2005', 'lsty':'-.', 'eva':0.4},
        2017 : {'years':[2017], 'color':'m', 'label':'2017', 'lsty':'-.', 'eva':0.4},
        2018 : {'years':[2018], 'color':'b', 'label':'2018', 'lsty':'-.', 'eva':0.4},
    }
    indicator_dict = {
        'ACE225' : {'name':'ACE','label':'ACE', 'threshold':225},
        # 'ACE159.6' : {'name':'ACE','label':'ACE', 'threshold':159.6},
        # 'ACE126.1' : {'name':'ACE','label':'ACE', 'threshold':126.1},
        # 'MajHur' : {'name':'MajHur', 'label':'major hurricanes', 'threshold':4.5},
        # 'Hur' : {'name':'Hur','label':'hurricanes', 'threshold':9.2},
    }

    ##################################
    # CDFs with difference highlight #
    ##################################
    for ind,indDT in indicator_dict.items():
        indicator = indDT['name']
        threshold = indDT['threshold']
        tmp = np.concatenate([d[indicator].values.flatten() for d in counterFacts.values()])
        X = np.linspace(tmp.min(),tmp.max(),1000)
        bw = (tmp.max() - tmp.min()) / 20
        fig,ax = plt.subplots(nrows=1, ncols=1, figsize=(4,3))
        # ax.set_title(indicator_name)
        for threshold in [225]:
            ax.axvline(x=threshold, color='gray')
            ax.annotate(threshold, xy=(threshold, 0), rotation=90, ha='center', va='bottom', backgroundcolor='w')
        intersects = pd.DataFrame()
        for yrName,yrDT in year_groups.items():
            ys = []
            for scen in ['%s SST levels' %(yr) for yr in [2018,1982,1900]]:
                DT = counterFacts[scen]
                tmp = DT[indicator][np.isin(DT[indicator].year,yrDT['years'])].values.reshape(-1, 1)
                kde = KernelDensity(kernel='gaussian', bandwidth=bw).fit(tmp)
                log_dens = kde.score_samples(X.reshape(-1, 1))
                y = np.exp(log_dens) / np.exp(log_dens).sum()
                ax.plot(X, np.cumsum(y), color=yrDT['color'], linestyle=DT['lsty'], zorder=10)
                ys.append(np.cumsum(y))

            ax.fill_between(X, ys[0], ys[2], color=yrDT['color'], alpha=0.3, zorder=10)

        for scen in ['%s SST levels' %(yr) for yr in [2018,1982,1900]]:
            DT = counterFacts[scen]
            ax.plot([],[],color='k', linestyle=DT['lsty'], label=DT['label'])

        for yrName,yrDT in year_groups.items():
            ax.plot([],[],color=yrDT['color'], label=yrDT['label'])

        ax.set_xlabel(indDT['label'])
        ax.legend(fontsize=6)

        if ind == 'ACE225':
            ax.annotate('A', xy=(-0.1,1.1), xycoords='axes fraction', fontweight='bold', backgroundcolor='w', ha='left', va='top', fontsize=12)
        plt.savefig(emu._dir+'/'+emu._tag+'/counter_2009-2018/'+indicator+'_cdf_thresh'+str(threshold)+'.png', bbox_inches='tight', dpi=400); plt.close()



In [21]:
    ############
    # FAR 2017 #
    ############
    # year_groups.pop(2018)

    N = 1000
    for ind,indDT in indicator_dict.items():
        indicator = indDT['name']
        threshold = indDT['threshold']
        fig,ax = plt.subplots(nrows=1, ncols=1, figsize=(5,3))
        # ax.set_title(indicator)
        ref = counterFacts['%s SST levels' %(2018)][indicator]
        for scen in ['%s SST levels' %(yr) for yr in [1982,1900]]:
            DT = counterFacts[scen]
            counter = DT[indicator]
            for yrName,yrDT in year_groups.items():
                yrs = yrDT['years']
                FAR,IR = [],[]
                for i in range(N):
                    probRef = np.sum(np.random.choice(ref[np.isin(ref.year,yrs)].values.flatten(), size=int(0.5*N), replace=True) >= threshold) / (0.5*N)
                    probCounter = np.sum(np.random.choice(counter[np.isin(counter.year,yrs)].values.flatten(), size=int(0.5*N), replace=True) >= threshold) / (0.5*N)
                    FAR.append(1 - probCounter / probRef)
                    IR.append(probRef / probCounter)
                if '1982' in scen:
                    ax.hist(FAR, bins=np.linspace(0,1,20), density=True, linestyle=DT['lsty'], color=yrDT['color'], alpha=0.5)

                probRef = np.sum(ref[np.isin(ref.year,yrs)].values.flatten() >= threshold) / len(ref[np.isin(ref.year,yrs)].values.flatten())
                probCounter = np.sum(counter[np.isin(ref.year,yrs)].values.flatten() >= threshold) / len(counter[np.isin(ref.year,yrs)].values.flatten())
                FAR = 1 - probCounter / probRef
                print('%s %s FAR %s' %(ind,yrName, FAR*100))
                ax.axvline(x=FAR, linestyle=DT['lsty'], color=yrDT['color'])#, label=yrDT['label'])

        ax.plot([],[],color='w', label='counterfactual scenario')
        for scen in ['%s SST levels' %(yr) for yr in [1982,1900]]:
            DT = counterFacts[scen]
            ax.plot([],[],color='k', linestyle=DT['lsty'], label=DT['label'])

        ax.plot([],[],color='w', label='weather conditions')
        for yrName,yrDT in year_groups.items():
            ax.plot([],[],color=yrDT['color'], label=yrDT['label'])

        ax.set_xlabel('Fraction of risk attributable to SST warming')
        ax.set_ylabel('estimated likelihood (normalized)')
        ax.set_ylim(0,5)
        ax2 = ax.twiny()
        ax2.set_xticks([0.5,0.75,0.9])
        ax2.set_xticklabels(['%sx' %(i) for i in [2,4,10]])
        ax2.set_xlabel('Increase in risk')
        ax.legend(fontsize=6, bbox_to_anchor=(1.05, 1), loc='upper left')
        if ind == 'ACE225':
            ax.annotate('B', xy=(-0.1,1.2), xycoords='axes fraction', fontweight='bold', backgroundcolor='w', ha='left', va='top', fontsize=12)
        plt.savefig(emu._dir+'/'+emu._tag+'/counter_2009-2018/'+indicator+'_far_special_thresh'+str(threshold)+'.png', bbox_inches='tight', dpi=200); plt.close()


ACE225 0 FAR 29.59381044487428

ACE225 2005 FAR 28.387096774193544

ACE225 2017 FAR 18.604651162790688

ACE225 0 FAR 68.66537717601547

ACE225 2005 FAR 62.58064516129031

ACE225 2017 FAR 65.11627906976744

In [66]:
print('prob of 225 ACE')
for scen in ['%s SST levels' %(yr) for yr in [0,2018,1982,1900]]:
    DT = counterFacts[scen]
    tmp = DT['ACE']
    print('%s : %s' %(scen, np.sum(tmp > 225).values / tmp.values.flatten().shape[0] * 100))


prob of 225 ACE

0 SST levels : 2.372972972972973

2018 SST levels : 2.7918918918918916

1982 SST levels : 1.9675675675675675

1900 SST levels : 0.872972972972973

In [64]:
print('prob of 225 ACE with 2017 conditions')
for scen in ['%s SST levels' %(yr) for yr in [0,2018,1982,1900]]:
    DT = counterFacts[scen]
    tmp = DT['ACE'].loc[2017]
    print('%s : %s' %(scen, np.sum(tmp > 225).values / tmp.values.flatten().shape[0] * 100))


prob of 225 ACE with 2017 conditions

0 SST levels : 3.0

2018 SST levels : 4.2

1982 SST levels : 3.5000000000000004

1900 SST levels : 1.5

In [67]:
print('prob of 225 ACE with 2005 conditions')
for scen in ['%s SST levels' %(yr) for yr in [0,2018,1982,1900]]:
    DT = counterFacts[scen]
    tmp = DT['ACE'].loc[2005]
    print('%s : %s' %(scen, np.sum(tmp > 225).values / tmp.values.flatten().shape[0] * 100))


prob of 225 ACE with 2005 conditions

0 SST levels : 14.000000000000002

2018 SST levels : 15.5

1982 SST levels : 11.1

1900 SST levels : 5.800000000000001